# Detectando objetos em videos com YOLOv4

## Etapa 1 - Download do Darknet

In [1]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.15 MiB | 13.03 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.


In [2]:
%cd darknet

/content/darknet


In [3]:
ls

3rdparty/               darknet_video.py        net_cam_v4.sh*
build/                  data/                   README.md
build.ps1*              image_yolov3.sh*        results/
cfg/                    image_yolov4.sh*        scripts/
cmake/                  include/                src/
CMakeLists.txt          json_mjpeg_streams.sh*  vcpkg.json
DarknetConfig.cmake.in  LICENSE                 video_yolov3.sh*
darknet_images.py       Makefile                video_yolov4.sh*
darknet.py              net_cam_v3.sh*


## Etapa 2 - Compilando a biblioteca

In [4]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [6]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |

## Etapa 3 - Carregando os pesos do modelo pré-treinado

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [8]:
import zipfile
path = "/content/gdrive/My Drive/YOLO/modelo_YOLOv4.zip"
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

## Etapa 4 - Carregando o vídeo

### 4.1 - De uma url

In [15]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4

--2023-01-28 00:10:45--  https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_teste02.mp4 [following]
--2023-01-28 00:10:46--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_teste02.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7266878 (6.9M) [application/octet-stream]
Saving to: ‘video_teste02.mp4.1’

video_teste02.mp4.1 100%[===================>]   6.93M  --.-KB/s    in 0.02s   

2023-01-28 00:10:48 (369 MB/s) - ‘video_teste02.mp4.1’ saved [7266878/7266878]



### 4.2 - Do Google Drive


In [17]:
!cp /content/gdrive/My\ Drive/YOLO/video_teste02.mp4 ./

cp: cannot stat '/content/gdrive/My Drive/YOLO/video_teste02.mp4': No such file or directory


In [18]:
ls

3rdparty/               darknet_video.py        README.md
backup/                 data/                   resultado.avi
build/                  image_yolov3.sh*        results/
build.ps1*              image_yolov4.sh*        scripts/
cfg/                    include/                src/
cmake/                  json_mjpeg_streams.sh*  vcpkg.json
CMakeLists.txt          LICENSE                 video_teste02.mp4
darknet*                Makefile                video_teste02.mp4.1
DarknetConfig.cmake.in  net_cam_v3.sh*          video_yolov3.sh*
darknet_images.py       net_cam_v4.sh*          video_yolov4.sh*
darknet.py              obj/                    yolov4.weights


## Etapa 5 - Realizando a detecção em vídeo

In [19]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_teste02.mp4 -i 0 -out_filename resultado.avi

A saída de streaming foi truncada nas últimas 5000 linhas.
traffic light: 38% 
traffic light: 27% 
truck: 53% 
car: 61% 
truck: 31% 
car: 100% 
car: 100% 
car: 96% 
car: 94% 
car: 93% 
car: 93% 
car: 92% 
car: 85% 
car: 44% 

FPS:18.8 	 AVG_FPS:18.7

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 92% 
traffic light: 56% 
traffic light: 32% 
truck: 61% 
car: 53% 
truck: 31% 
car: 88% 
car: 100% 
car: 100% 
car: 97% 
car: 97% 
car: 94% 
car: 94% 
car: 86% 
car: 50% 
car: 28% 

FPS:18.8 	 AVG_FPS:18.7

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 93% 
traffic light: 64% 
traffic light: 44% 
truck: 58% 
car: 52% 
car: 100% 
car: 100% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 95% 
car: 91% 
car: 39% 

FPS:18.8 	 AVG_FPS:18.7

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 93% 
traffic light: 64% 
traffic light: 44% 
truck: 58% 
car: 51% 
car: 100% 
car: 100% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 95% 
car: 91% 
car: 39% 

FPS:18.8 	 AVG_FP

In [20]:
!du -h resultado.avi

32M	resultado.avi


## Etapa 6 - Visualizando o resultado

In [21]:
!cp ./resultado.avi /content/gdrive/My\ Drive/YOLO/resultado1.avi

## Especificando um threshold

In [22]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4

--2023-01-28 00:12:14--  https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_rua01.mp4 [following]
--2023-01-28 00:12:15--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_rua01.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5645465 (5.4M) [application/octet-stream]
Saving to: ‘video_rua01.mp4’

video_rua01.mp4     100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2023-01-28 00:12:16 (256 MB/s) - ‘video_rua01.mp4’ saved [5645465/5645465]



In [23]:
!cp /content/gdrive/My\ Drive/YOLO/video_rua01.mp4 ./

cp: cannot stat '/content/gdrive/My Drive/YOLO/video_rua01.mp4': No such file or directory


In [24]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_rua01.mp4 -out_filename resultado02.avi -thresh 0.3

A saída de streaming foi truncada nas últimas 5000 linhas.
handbag: 41% 
traffic light: 39% 
car: 97% 
car: 85% 
car: 79% 
car: 58% 
car: 57% 
person: 99% 
person: 99% 
person: 98% 
person: 98% 
person: 97% 
person: 96% 
person: 95% 
person: 80% 
person: 52% 
person: 39% 
person: 32% 

FPS:13.9 	 AVG_FPS:13.5

 cvWriteFrame 
Objects:

handbag: 57% 
traffic light: 39% 
car: 98% 
car: 89% 
car: 78% 
car: 58% 
car: 57% 
person: 99% 
person: 99% 
person: 99% 
person: 99% 
person: 96% 
person: 96% 
person: 94% 
person: 80% 
person: 53% 
person: 39% 
person: 33% 

FPS:13.7 	 AVG_FPS:13.5

 cvWriteFrame 
Objects:

traffic light: 37% 
car: 98% 
car: 94% 
car: 77% 
car: 56% 
car: 54% 
car: 32% 
person: 99% 
person: 99% 
person: 99% 
person: 98% 
person: 96% 
person: 96% 
person: 95% 
person: 81% 
person: 71% 
person: 50% 
person: 38% 
person: 31% 

FPS:13.1 	 AVG_FPS:13.5

 cvWriteFrame 
Objects:

traffic light: 39% 
car: 98% 
car: 95% 
car: 70% 
car: 50% 
car: 42% 
car: 32% 
person: 99% 
perso

In [25]:
!cp ./resultado02.avi /content/gdrive/My\ Drive/YOLO/resultado2.avi

In [27]:
!cp /content/gdrive/My\ Drive/YOLO/imagens/roadusa.mp4 ./

In [28]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show roadusa.mp4 -out_filename resultado03.avi -thresh 0.3

 CUDA-version: 11020 (11060), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 4.2.0
Demo
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1/ 1    3

In [29]:
!cp ./resultado03.avi /content/gdrive/My\ Drive/YOLO/resultado3.avi

In [30]:
!cp /content/gdrive/My\ Drive/YOLO/imagens/roadusa2.mp4 ./
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show roadusa2.mp4 -out_filename roadusa2.avi -thresh 0.3
!cp ./roadusa2.avi /content/gdrive/My\ Drive/YOLO/roadusa2.avi

A saída de streaming foi truncada nas últimas 5000 linhas.
Objects:

truck: 82% 
truck: 42% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

traffic light: 34% 
traffic light: 33% 
truck: 50% 
truck: 30% 
person: 49% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

traffic light: 33% 
truck: 50% 
truck: 31% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

traffic light: 32% 
truck: 44% 
truck: 37% 
person: 42% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

bird: 58% 
truck: 42% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

bird: 31% 
traffic light: 36% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

traffic light: 64% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

traffic light: 59% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

traffic light: 62% 

FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:


FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:


FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:


FPS:19.5 	 AVG_FPS:19.3

 cvWriteFrame 
Objects:

tra

In [32]:
!cp /content/gdrive/My\ Drive/YOLO/imagens/roadusa3.mp4 ./
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show roadusa3.mp4 -out_filename roadusa3.avi -thresh 0.3
!cp ./roadusa3.avi /content/gdrive/My\ Drive/YOLO/roadusa3.avi

 CUDA-version: 11020 (11060), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 4.2.0
Demo
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1/ 1    3